# Module 2: Foundations of Prompt Engineering

CONCEPT:
- Prompt Structure = ROLE + TASK + CONTEXT + CONSTRAINTS + FORMAT

Techniques:
- ZERO-SHOT: No examples given
- FEW-SHOT: Give examples first
- CHAIN-OF-THOUGHT: Step by step thinking
- DECOMPOSITION: Break into smaller parts

### Task 1: ZERO-SHOT 
##### Use payment_system.py

In [ ]:
# Prompt 1: 
"Generate 4 test cases for process_payment function covering positive and negative scenarios"

# What happens: Creates tests without examples

Answer: -
import pytest
from module_2.payment_system import process_payment

#### These tests assume payment_system calls an internal gateway function
#### (e.g. _call_gateway) which we mock. Adjust mock target to match your implementation.

def test_process_payment_success(monkeypatch):
    # Gateway returns a success payload
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda amount, card, cvv, expiry, **kw: {"status": "success", "transaction_id": "tx123"})
    result = process_payment(100.00, "4111111111111111", "123", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "success"
    assert "transaction_id" in result

def test_process_payment_invalid_card(monkeypatch):
    # Gateway indicates card invalid
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "error", "error_code": "invalid_card"})
    result = process_payment(50.00, "0000000000000000", "000", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "error"
    assert result.get("error_code") == "invalid_card"

def test_process_payment_insufficient_funds(monkeypatch):
    # Gateway declines due to insufficient funds
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "declined", "reason": "insufficient_funds"})
    result = process_payment(500.00, "4111111111111111", "123", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "declined"
    assert result.get("reason") == "insufficient_funds"

def test_process_payment_expired_card():
    # Validate local expiry check: expired card should be rejected before gateway call
    with pytest.raises(ValueError):
        process_payment(20.00, "4111111111111111", "123", "01/20")

In [ ]:
# Prompt 2: 
"List all edge cases for calculate_tax function and create test cases for each"

Answer:-
Edge cases (brief):

Typical country rates (e.g. US, UK)
Zero amount
Negative amount (invalid)
Unknown/unsupported country code
Amount provided as string vs numeric
Very large amounts
Rounding/precision for fractional amounts

### Task 2: FEW-SHOT

In [ ]:
# Prompt 1: 
"Example test case: test_valid_payment() tests amount=100, card=1234567890123456, cvv=123 expects Payment successful. Now create 3 similar test cases for invalid inputs"

# What happens: Follows your example pattern

Answer:- 

import pytest
from module_2.payment_system import process_payment

def test_invalid_card_number(monkeypatch):
    # Gateway returns invalid card error
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "error", "error_code": "invalid_card"})
    result = process_payment(100, "1234567890123456", "123", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "error"
    assert result.get("error_code") == "invalid_card"

def test_invalid_cvv(monkeypatch):
    # Gateway returns invalid CVV error
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "error", "error_code": "invalid_cvv"})
    result = process_payment(100, "4111111111111111", "12", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "error"
    assert result.get("error_code") == "invalid_cvv"

def test_expired_card_rejected():
    # Local validation should reject expired card before gateway call
    with pytest.raises(ValueError):
        process_payment(100, "4111111111111111", "123", "01/20")

In [ ]:
# Prompt 2: 
"Example: test_us_tax() tests amount=100, country=US, expects 10.0. Now create similar tests for UK and other countries"

Answer:-
Tests (assumed tax rates: US 10%, UK 20%, DE 19%, CA 5%, AU 10% — adjust to your implementation):

import pytest
from module_2.tax import calculate_tax  # adjust import to match your code

def test_calculate_tax_uk():
    # UK VAT 20% -> tax = 20.0 for 100
    assert calculate_tax(100, "UK") == pytest.approx(20.0)

def test_calculate_tax_germany():
    # Germany VAT 19% -> tax = 19.0 for 100
    assert calculate_tax(100, "DE") == pytest.approx(19.0)

def test_calculate_tax_canada():
    # Canada GST 5% -> tax = 5.0 for 100
    assert calculate_tax(100, "CA") == pytest.approx(5.0)

def test_calculate_tax_australia():
    # Australia GST 10% -> tax = 10.0 for 100
    assert calculate_tax(100, "AU") == pytest.approx(10.0)

### Task 3: CHAIN-OF-THOUGHT

In [ ]:
# Prompt 1: 
"For process_payment function, first list all validation rules, then identify what inputs would break each rule, then generate test cases for each scenario"

# What happens: Thinks step by step

Answer:-

Validation rules (short):

Amount: numeric and > 0.
Card number: digits, correct length/Luhn (or handled by gateway).
CVV: numeric and correct length (3 or 4).
Expiry: valid format and not expired (local check).
Input types: parameters coercible to expected types (no None, strings where numbers expected).
Gateway responses: handle success, error codes, declines, and gateway exceptions.
Inputs that break each rule (examples):

Amount not numeric ("100$", None) or <= 0 (0, -5).
Card contains letters or wrong length ("1234abcd", "0000") or fails Luhn.
CVV non-numeric ("12a") or wrong length ("12", "12345").
Expiry in past ("01/20"), malformed ("202012").
None or wrong-type args (None, object()).
Gateway returns {"status":"error","error_code":...} or {"status":"declined",...} or raises exceptions.


import pytest
from module_2.payment_system import process_payment

def test_amount_not_numeric_raises_typeerror():
    with pytest.raises(TypeError):
        process_payment("one hundred", "4111111111111111", "123", "12/30")

def test_amount_negative_raises_valueerror():
    with pytest.raises(ValueError):
        process_payment(-10.00, "4111111111111111", "123", "12/30")

def test_expired_card_rejected():
    # Local expiry validation should raise before contacting gateway
    with pytest.raises(ValueError):
        process_payment(20.00, "4111111111111111", "123", "01/20")

def test_gateway_invalid_card_returns_error(monkeypatch):
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "error", "error_code": "invalid_card"})
    result = process_payment(50.00, "0000000000000000", "000", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "error"
    assert result.get("error_code") == "invalid_card"

def test_gateway_insufficient_funds_decline(monkeypatch):
    monkeypatch.setattr("module_2.payment_system._call_gateway",
                        lambda *args, **kwargs: {"status": "declined", "reason": "insufficient_funds"})
    result = process_payment(500.00, "4111111111111111", "123", "12/30")
    assert isinstance(result, dict)
    assert result.get("status") == "declined"
    assert result.get("reason") == "insufficient_funds"

In [ ]:

# Prompt 2: 
"For calculate_tax function, first identify all country codes, then determine tax rate for each, then create test cases with calculations"

Answer:-

Country codes and assumed tax rates (adjust to your implementation):

US: 10%
UK: 20%
DE: 19%
CA: 5%
AU: 10%

import pytest
from module_2.tax import calculate_tax  # adjust import if different

def test_calculate_tax_us():
    # US 10% -> tax = 10.0 for 100
    assert calculate_tax(100, "US") == pytest.approx(10.0)

def test_calculate_tax_uk():
    # UK 20% -> tax = 20.0 for 100
    assert calculate_tax(100, "UK") == pytest.approx(20.0)

def test_calculate_tax_germany():
    # DE 19% -> tax = 19.0 for 100
    assert calculate_tax(100, "DE") == pytest.approx(19.0)

def test_calculate_tax_canada():
    # CA 5% -> tax = 5.0 for 100
    assert calculate_tax(100, "CA") == pytest.approx(5.0)

def test_calculate_tax_australia():
    # AU 10% -> tax = 10.0 for 100
    assert calculate_tax(100, "AU") == pytest.approx(10.0)

### Task 4: DECOMPOSITION

In [ ]:
# Prompt 1: 
"Step 1: Identify all input parameters in process_payment. Step 2: For each parameter, list valid and invalid values. Step 3: Create one boundary test case for each parameter"

# What happens: Handles each step separately

Answer:- 

import pytest
from module_2.payment_system import process_payment

def test_amount_zero_raises_value_error():
    # Boundary between invalid and valid amount (0 is invalid)
    with pytest.raises(ValueError):
        process_payment(0, "4111111111111111", "123", "12/30")

def test_card_length_too_short_raises_value_error():
    # Boundary: 15 digits (too short if 16 is required)
    with pytest.raises(ValueError):
        process_payment(10.0, "411111111111111", "123", "12/30")

def test_cvv_too_short_raises_value_error():
    # Boundary: 2-digit CVV (too short if 3 is minimum)
    with pytest.raises(ValueError):
        process_payment(10.0, "4111111111111111", "12", "12/30")

def test_expiry_malformed_raises_value_error():
    # Boundary: malformed expiry string should be rejected
    with pytest.raises(ValueError):
        process_payment(10.0, "4111111111111111", "123", "202012")

In [ ]:

# Prompt 2: 
"Step 1: List all possible country values in calculate_tax. Step 2: Identify the tax rate for each. Step 3: Generate test with amount=1000 for each country"


Answer:-

Country values and assumed tax rates:

US: 10%
UK: 20%
DE: 19%
CA: 5%
AU: 10%

import pytest
from module_2.tax import calculate_tax  # adjust import if different

def test_calculate_tax_us_1000():
    assert calculate_tax(1000, "US") == pytest.approx(100.0)

def test_calculate_tax_uk_1000():
    assert calculate_tax(1000, "UK") == pytest.approx(200.0)

def test_calculate_tax_de_1000():
    assert calculate_tax(1000, "DE") == pytest.approx(190.0)

def test_calculate_tax_ca_1000():
    assert calculate_tax(1000, "CA") == pytest.approx(50.0)

def test_calculate_tax_au_1000():
    assert calculate_tax(1000, "AU") == pytest.approx(100.0)

### Task 5: ROLE-BASED

In [ ]:
# Prompt 1: 
"Act as a security tester. Review process_payment function and identify security vulnerabilities. List 3 security test cases"

# What happens: Responds from specific role perspective


Answer:- 

import pytest
from module_2 import payment_system
from module_2.payment_system import process_payment

def test_no_sensitive_logging(monkeypatch):
    """Ensure card/CVV are not written to logs."""
    logs = []
    class FakeLogger:
        def info(self, msg, *a, **k): logs.append(str(msg))
        def warning(self, msg, *a, **k): logs.append(str(msg))
        def error(self, msg, *a, **k): logs.append(str(msg))
    monkeypatch.setattr(payment_system, "logger", FakeLogger())
    # Mock gateway to succeed so only logging behavior is exercised
    monkeypatch.setattr(payment_system, "_call_gateway", lambda *a, **k: {"status": "success", "transaction_id": "tx1"})
    process_payment(100, "1234567890123456", "123", "12/30")
    assert not any("1234567890123456" in m or "123" in m for m in logs)

def test_handles_gateway_exception_without_leaking_sensitive_info(monkeypatch):
    """Gateway exceptions must be handled and must not leak internal secrets."""
    def raise_ex(*a, **k):
        raise RuntimeError("db_password=SECRET_VALUE")
    monkeypatch.setattr(payment_system, "_call_gateway", raise_ex)
    # Call should not propagate internal exception or include secret in returned data
    result = process_payment(50, "4111111111111111", "999", "12/30")
    assert not isinstance(result, Exception)
    assert "SECRET_VALUE" not in repr(result).upper()

def test_rejects_malicious_input_patterns():
    """Reject obvious injection/malicious patterns in inputs."""
    with pytest.raises(ValueError):
        process_payment(10, "4111' OR '1'='1", "123", "12/30")

In [ ]:
# Prompt 2: 
"Act as a performance tester. Identify performance concerns in calculate_tax function and suggest 2 performance test scenarios"


Answer:-

Performance concerns (brief)

Per-call latency: expensive lookups, regex/parsing, or remote rate fetch can increase response time.
Throughput / concurrency: contention if rates cached with locks or if I/O blocks.
Memory / allocations: handling very large lists or large numeric types can spike memory.
Worst-case code paths: exceptions, fallback logic, or string->number coercion may be slower.
Precision/decimal work: using Decimal vs float may add CPU overhead.
Two performance test scenarios (measurable, actionable)

High-throughput stress — measure avg latency and throughput
Goal: ensure calculate_tax can process N calls without exceeding latency threshold.
Metrics: total time, avg time per call, operations/sec, failures.
Run: N = 100_000 mixed country calls; expect avg < X ms (set per SLA).

import pytest
from module_2.tax import calculate_tax

def test_calculate_tax_throughput(benchmark):
    countries = ["US", "UK", "DE", "CA", "AU"]
    calls = countries * 20000  # 100k calls total

    def run_all():
        for c in calls:
            calculate_tax(100.0, c)

    # benchmark the full run; interpret results: total time / 100k -> avg per call
    benchmark.pedantic(run_all, rounds=3, iterations=1)

### Task 6: STRUCTURED OUTPUT

In [ ]:
# Prompt 1: 
"For process_payment function generate test cases in this format: Test Name | Input | Expected Output | Test Type"


# What happens: Formats as specified structure

Answer:-

#### Test Name | Input | Expected Output | Test Type
Test_Process_Payment_Success | (amount=100.00, card="4111111111111111", cvv="123", expiry="12/30") | {"status":"success","transaction_id": "<non-empty string>"} | functional / positive
Test_Invalid_Card_Number | (amount=50.00, card="0000000000000000", cvv="000", expiry="12/30") | {"status":"error","error_code":"invalid_card"} | negative / validation
Test_Insufficient_Funds | (amount=500.00, card="4111111111111111", cvv="123", expiry="12/30") | {"status":"declined","reason":"insufficient_funds"} | negative / gateway
Test_Expired_Card | (amount=20.00, card="4111111111111111", cvv="123", expiry="01/20") | raises ValueError (reject before gateway) | negative / validation
Test_Amount_Zero_Boundary | (amount=0.00, card="4111111111111111", cvv="123", expiry="12/30") | raises ValueError | boundary
Test_Invalid_CVV | (amount=100.00, card="4111111111111111", cvv="12", expiry="12/30") | raises ValueError or {"status":"error","error_code":"invalid_cvv"} (depending on local vs gateway validation) | negative / validation
Test_Gateway_Exception_Handled | (amount=100.00, card="4111111111111111", cvv="123", expiry="12/30") + gateway raises exception | returns sanitized error (no exception propagated), no secrets leaked | negative / resilience
Test_No_Sensitive_Logging | (amount=100.00, card="1234567890123456", cvv="123", expiry="12/30") | application logs must NOT contain full card number or CVV | security / privacy

In [ ]:
# Prompt 2: 
"For calculate_tax function create test matrix with columns: Country | Amount | Expected Tax | Pass/Fail Criteria"

Answer:- 

Using the implementation in module_2.payment_system.calculate_tax (US: 10%, UK: 20%, other: 5%).


Country	Amount	Expected Tax	Pass/Fail Criteria
US	1000	100.0	calculate_tax(1000, "US") == pytest.approx(100.0)
UK	1000	200.0	calculate_tax(1000, "UK") == pytest.approx(200.0)
DE (other)	1000	50.0	calculate_tax(1000, "DE") == pytest.approx(50.0)
CA (other)	1000	50.0	calculate_tax(1000, "CA") == pytest.approx(50.0)
AU (other)	1000	50.0	calculate_tax(1000, "AU") == pytest.approx(50.0)
Unknown (e.g., "ZZ")	1000	50.0	calculate_tax(1000, "ZZ") == pytest.approx(50.0)


import pytest
from module_2.payment_system import calculate_tax

@pytest.mark.parametrize("country,amount,expected", [
    ("US", 1000, 100.0),
    ("UK", 1000, 200.0),
    ("DE", 1000, 50.0),
    ("CA", 1000, 50.0),
    ("AU", 1000, 50.0),
    ("ZZ", 1000, 50.0),
])
def test_calculate_tax_matrix(country, amount, expected):
    assert calculate_tax(amount, country) == pytest.approx(expected)
```# filepath: c:\Users\admin\OneDrive - Blue Data Consulting\Prompt Engineering\module_2\tests\test_calculate_tax_matrix.py
import pytest
from module_2.payment_system import calculate_tax

@pytest.mark.parametrize("country,amount,expected", [
    ("US", 1000, 100.0),
    ("UK", 1000, 200.0),
    ("DE", 1000, 50.0),
    ("CA", 1000, 50.0),
    ("AU", 1000, 50.0),
    ("ZZ", 1000, 50.0),
])
def test_calculate_tax_matrix(country, amount, expected):
    assert calculate_tax(amount, country) == pytest.approx(expected)